# Table of contents 
1. [Notation](#notation)
2. [Pseudo Code](#pseudo)
3. [Import Packages](#import_models)
4. [Model](#model)


## Notation <a name="notation"></a>

- $A_0(n,p)$ is the assessment of item $n$ for patient $p$ at the beginning of the treatment (timepoint 0). This is a numerical value: 0, 1, 2, or 3, with a higher number indicating a greater need.
- $A_1(n,p)$ is the assessment of item $n$ for patient $p$ at the end of the treatment (timepoint 1). This is a numerical value: 0, 1, 2, or 3, with a higher number indicating a greater need.

## Pseudo Code <a name="pseudo"></a>

+ For iteration **i** in $\{1,2,3,...iter_num\}$:
  1. We have history data set with *`IBHS_num`* patients who received IBHS, and *`FBMHS_num`* patients who received FBMHS, and *`num_items`* items. Next, we randomly divide the data with each treatment into *`batch_num`* batches with 
  
      - `ibhs_batch_size` = floor(*`IBHS_num`*/*`batch_num`*)  (except one group with size = *`IBHS_num`*  - *`batch_num`* $\times$ `ibhs_batch_size`)
      - `fbmhs_batch_size` = floor(*`FBMHS_num`*/*`batch_num`*)  (except one group with size = *`FBMHS_num`*  - *`batch_num`* $\times$ `fbmhs_batch_size`)
  1. For batch $j$ in $\{1,2,3,..., $*`batch_num`*$\}$: 
      1. Consider $ibhs\_batch_j$, $fbmhs\_batch_j$
      2. We initiate item $x^{i}_{nj} \in[-1,1]$ for iteration **i**, item **n** and batch **j**, where $-1$ means the item improves with IBHS and $1$ means the item improves with FBMHS. 0 means item either improves with both treatments equally or does not improve with any treatments.
      3. For each patient $p$ in $ibhs\_batch_j$ and $fbmhs\_batch_j$, define patient score $x^{i}_{pj}$ for iteration **i**, patient **p** in batch **j**
      $$x^{i}_{pj} = \frac{1}{3\cdot (\text{num_items})}\sum_{n\in num\_items} x^{i}_{nj}  \cdot (A_0(n,p) - A_1(n,p)) \in [-1,1]$$ 
      where $(A_0(n,p) - A_1(n,p)) \in \{0,1,2,3\}$ is the intial score of item $n$ for patient $p$ - final score of item $n$ for patient $p$
      
      **Objective:**
      $$max\frac{1}{|\text{ibhs_batch_size}|} \sum_{p \in ibhs\_batch_{j}}x^{i}_{pj}\cdot (-1)\cdot I(p) + \frac{1}{|\text{fbmhs_batch_size}|}\sum_{p \in fbmhs\_batch_{j}}x^{i}_{pj} \cdot 1 \cdot I(p)$$ 
      where $I(p)$ is the cumulative improvement for patient $p$.   $I(p) = \sum_{n \in num\_items} A_0(n,p)-A_1(n,p)$
      4. We use Gurobi to calculate step C, and we get $x^{i}_{1j}, x^{i}_{2j}, ...$ for all items of batch **j** in iteration **i**  
  2. We calcualte the average score of each item **n** for iteration **i**:
      $$x^{i}_{n} = \frac{1}{batch\_num}\sum_{j} x^{i}_{nj}$$
     
+ Lastly, we find iteration average $x_{n}$  for each item $n$
    $$x_{n} = \frac{1}{iter\_num}\sum_{i} x^{i}_{n}$$



## Import Packages <a name="import_models"></a>

In [10]:
#import the modules
from gurobipy import *
import gurobipy as gp
from gurobipy import GRB
import os
import numpy as np
import pandas as pd
import csv 
import random
import sys
import matplotlib.pyplot as plt

In [11]:
#Some useful settings
#np.set_printoptions(threshold=sys.maxsize)
# %pprint
#pd.set_option('display.max_rows', None)
#pd.set_option('display.max_rows', 10)
#pd.set_option('display.max_columns', None)

### Model<a name="model"></a>

In [51]:
def twosum_improvement_xneed_once(IBHS, FBMHS, batch_num):  
    
    IBHS_num = IBHS.shape[0]
    FBMHS_num = FBMHS.shape[0]
    num_items = IBHS.shape[1]
    #print('there are ', str(num_items), ' needs.')
    #print('there are ', str(IBHS_num), ' IBHS Patients.')
    #print('there are ', str(FBMHS_num), ' FBMHS Patients.')

    FBMHS = FBMHS.to_numpy()
    selected_rows = np.random.choice(FBMHS_num, size=FBMHS_num, replace=False)
    FBMHS = FBMHS[selected_rows, :]
    
    IBHS = IBHS.to_numpy()
    selected_rows = np.random.choice(IBHS_num, size=IBHS_num, replace=False)
    IBHS = IBHS[selected_rows, :]
    #print(selected_rows)
    ibhs_batch_size = IBHS_num//batch_num
    fbmhs_batch_size = FBMHS_num//batch_num
    
    ibhs_batch = None
    fbmhs_batch = None
     
    res = np.zeros((batch_num, num_items))
    
    for k in range(batch_num):        
        if k < batch_num-1:
            ibhs_batch = IBHS[k*ibhs_batch_size:(k+1)*ibhs_batch_size,]
            fbmhs_batch = FBMHS[k*fbmhs_batch_size:(k+1)*fbmhs_batch_size,]
        else:
            ibhs_batch = IBHS[k*ibhs_batch_size:,]
            fbmhs_batch = FBMHS[k*fbmhs_batch_size:,]

        #Gurobi model                
        m = gp.Model()
        m.Params.LogToConsole = 0 #Gurobi would not print

        #Create decision variables
        x_need = m.addVars(num_items, lb = -1, ub = 1, vtype = GRB.CONTINUOUS, name = 'x_need')

        #Find x weighted score
        x_patient_IBHS = [0 for i in range(ibhs_batch.shape[0])]
        for i in range(ibhs_batch.shape[0]):
            x_patient_IBHS[i] = sum(x_need[j]*ibhs_batch[i][j] for j in range(num_items))/(3*num_items)

        x_patient_FBMHS = [0 for i in range(fbmhs_batch.shape[0])]
        for i in range(fbmhs_batch.shape[0]):
            x_patient_FBMHS[i] = sum(x_need[j]*fbmhs_batch[i][j] for j in range(num_items))/(3*num_items)


        #Using given data to find total improvement for each patient
        x_improvement_IBHS = [0 for i in range(ibhs_batch.shape[0])]
        x_improvement_FBMHS = [0 for i in range(fbmhs_batch.shape[0])]

        for i in range(ibhs_batch.shape[0]):
            x_improvement_IBHS[i] = sum(ibhs_batch[i][j] for j in range(num_items))

        for i in range(fbmhs_batch.shape[0]):
            x_improvement_FBMHS[i] = sum(fbmhs_batch[i][j] for j in range(num_items))

        ibhs_sum = gp.quicksum(x_patient_IBHS[i]*(-1)*x_improvement_IBHS[i] for i in range(ibhs_batch.shape[0]))
        fbmh_sum = gp.quicksum(x_patient_FBMHS[i]*x_improvement_FBMHS[i] for i in range(fbmhs_batch.shape[0]))
        m.setObjective(ibhs_sum/(ibhs_batch.shape[0]) + fbmh_sum/fbmhs_batch.shape[0],
                       GRB.MAXIMIZE)

        m.optimize()
        
        res[k,:] = [x_need[i].X for i in range(num_items)]
        
    return np.sum(res, axis = 0)/batch_num

In [52]:
def twosum_improvement_xneed_all(IBHS, FBMHS, batch_num = 10, n = 100):
    num_items = IBHS.shape[1]
    total = np.array([0.0 for i in range(num_items)])
    for i in range(n):
        xneed_real = twosum_improvement_xneed_once(IBHS, FBMHS, batch_num)
        total += xneed_real
    final_need = total/n
    return final_need